In [1]:
# -*- coding: utf-8 -*-
import re, time, logging, gensim
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn import linear_model

c:\program files\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
%%time
def testCorpus(path_, split = 100):
    print('loading corpus')
    result, spl = [], 0
    file = open(path_, 'r', encoding='utf8')
    for line in file:
        if spl != split:
            element = line.split(',')
            result.append([element[0], element[1], element[2], int(element[3]), int(element[4]), int(element[5])])
        else:
            break
    print('test corpus loaded!')
    return(result)

test_corpus = testCorpus('validation_wiki.txt')

loading corpus
test corpus loaded!
Wall time: 10 ms


In [3]:
%%time
path_ = 'glove.6B.200d.txt'
word_vectors = KeyedVectors.load_word2vec_format(path_, binary=False)

Wall time: 1min 54s


In [4]:
%%time
X, y = [], []
for group in test_corpus:
    try:
        sim_01 = word_vectors.similarity(group[0], group[1])
        sim_12 = word_vectors.similarity(group[1], group[2])
        sim_02 = word_vectors.similarity(group[0], group[2])
        wiki_3_in_1 = group[4]
        wiki_3_in_2 = group[5]
        if word_vectors.doesnt_match(group[:3]) == group[2]:
            third_wheel = 1
        else:
            third_wheel = 0
            
        #print(group[0], group[1], group[2], sim_01, sim_12, sim_02, wiki_3_in_1, wiki_3_in_2, third_wheel)
        
        X.append((sim_01, sim_12, sim_02, wiki_3_in_1, wiki_3_in_2, third_wheel))
        y.append(group[3])
        
    except:
        pass
    
X_train = X[:-1000]
X_test = X[-1000:]
y_train = y[:-1000]
y_test = y[-1000:]

Wall time: 1.32 s


In [5]:
clf_logreg = linear_model.LogisticRegression(random_state = 33)
clf_logreg.fit(X_train, y_train)
predictions = clf_logreg.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.66      0.58      0.62       509
          1       0.61      0.69      0.65       491

avg / total       0.64      0.63      0.63      1000

